In [1]:
import pandas as pd
import numpy as np

In [2]:
tr_pt=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w7/Transaction Path.csv')
acc_hol=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w7/Account Holders.csv')
acc_inf=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w7/Account Information.csv')
tr_det=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w7/Transaction Detail.csv')

In [3]:
acc_hol.dtypes

Account Holder ID         int64
Name                     object
Date of Birth            object
Contact Number            int64
First Line of Address    object
dtype: object

In [4]:
acc_hol['Contact Number']=acc_hol['Contact Number'].astype('str').apply(lambda x:'0'+x)
acc_hol['Account Holder ID']=acc_hol['Account Holder ID'].astype('str')
acc_hol.head(6)

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park
1,20123998,Maynord Surgeoner,21/08/1997,07716107305,6422 Buena Vista Plaza
2,54374080,Giraldo Kimbley,22/03/1995,07489940612,93005 Summer Ridge Avenue
3,97027297,Blake Dudson,30/06/1955,07253587445,2 Huxley Hill
4,89920386,Ajay Douce,19/12/1930,07395580534,90176 Miller Alley
5,97325900,Evonne Lindell,07/07/1946,07631255412,90176 Miller Alley


In [5]:
acc_inf=acc_inf.set_index(['Account Number','Account Type','Balance Date','Balance']).apply(lambda x: x.str.split(',').explode()).reset_index()
acc_inf.head(6)

,Account Number,Account Type,Balance Date,Balance,Account Holder ID
0,10005367,Platinum,2023-01-31,728.25,70390615
1,10011977,Basic,2023-01-31,676.54,20123998
2,10024680,Platinum,2023-01-31,567.46,54374080
3,10031238,Basic,2023-01-31,576.52,97027297
4,10034341,Joint,2023-01-31,390.39,89920386
5,10034341,Joint,2023-01-31,390.39,97325900


In [6]:
acc_inf_fin=pd.merge(acc_hol,acc_inf,how='inner',on='Account Holder ID')
acc_inf_fin['Account Number']=acc_inf_fin['Account Number'].astype('str')
acc_inf_fin.head(10)

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address,Account Number,Account Type,Balance Date,Balance
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25
1,20123998,Maynord Surgeoner,21/08/1997,07716107305,6422 Buena Vista Plaza,10011977,Basic,2023-01-31,676.54
2,54374080,Giraldo Kimbley,22/03/1995,07489940612,93005 Summer Ridge Avenue,10024680,Platinum,2023-01-31,567.46
3,97027297,Blake Dudson,30/06/1955,07253587445,2 Huxley Hill,10031238,Basic,2023-01-31,576.52
4,89920386,Ajay Douce,19/12/1930,07395580534,90176 Miller Alley,10034341,Joint,2023-01-31,390.39
5,89460366,Kayla Emtage,12/10/1987,07867994451,8 Green Ridge Alley,10079965,Basic,2023-01-31,799.27
6,53258828,Effie Bigglestone,04/03/1992,07623832140,231 Spenser Hill,10091076,Platinum,2023-01-31,831.83
7,41402857,Federico Cathro,13/12/1985,07283906757,12 Green Hill,10231977,Basic,2023-01-31,61.63
8,84620072,Kalvin Duetschens,27/03/1965,07727619510,19032 Duke Parkway,10243581,Gold,2023-01-31,104.27
9,36711164,Cordey Blincowe,22/02/1970,07691986749,24 Birchwood Place,10271519,Platinum,2023-01-31,50.02


In [7]:
tr_pt.dtypes

Transaction ID    int64
Account_To        int64
Account_From      int64
dtype: object

In [8]:
tr_det.dtypes

Transaction ID        int64
Transaction Date     object
Value               float64
Cancelled?           object
dtype: object

In [9]:
tr_fin=pd.merge(tr_pt,tr_det,how='inner',on='Transaction ID')
tr_fin=tr_fin[(tr_fin['Cancelled?']=='N')&(tr_fin['Value']>1000)]
tr_fin['Account_From']=tr_fin['Account_From'].astype('str')
#tr_fin=tr_fin.rename(columns={'Account_From':'Account Holder ID'})
tr_fin

,Transaction ID,Account_To,Account_From,Transaction Date,Value,Cancelled?
23,45024251,15826579,53727603,2023-02-11,1137.9,N
24,46779876,21694967,82023979,2023-02-02,1019.5,N
35,57491925,19094015,65589565,2023-02-12,1761.1,N
81,108536548,86207903,10553619,2023-02-06,1085.4,N
131,176916326,33632099,18185362,2023-02-12,1955.6,N
...,...,...,...,...,...,...
8645,9849651040,19789505,88160468,2023-02-01,1041.9,N
8663,9859809917,25678043,24529850,2023-02-08,2283.2,N
8680,9881869013,41252281,53845615,2023-02-10,1052.9,N
8696,9896902976,26070761,93747938,2023-02-07,1076.8,N


In [21]:
sol=pd.merge(tr_fin,acc_inf_fin,how='inner',left_on='Account_From',right_on='Account Number')
sol=sol[sol['Account Type']!= "Platinum"]

In [28]:
sol_fin=sol[['Transaction ID','Account_To','Transaction Date','Value','Account Number','Account Type',
             'Balance Date','Balance','Name','Date of Birth','Contact Number','First Line of Address']]
sol_fin.head(10)

,Transaction ID,Account_To,Transaction Date,Value,Account Number,Account Type,Balance Date,Balance,Name,Date of Birth,Contact Number,First Line of Address
2,46779876,21694967,2023-02-02,1019.5,82023979,Gold,2023-01-31,744.37,Barb Eminson,20/07/1974,07748168711,3149 American Ash Hill
3,57491925,19094015,2023-02-12,1761.1,65589565,Gold,2023-01-31,321.99,Annabel Medd,17/02/1959,07175713923,24 Warner Junction
4,108536548,86207903,2023-02-06,1085.4,10553619,Basic,2023-01-31,756.20,Dud Gaunt,04/12/1983,07681115087,3 Canary Avenue
5,2843750091,20090784,2023-02-04,1263.9,10553619,Basic,2023-01-31,756.20,Dud Gaunt,04/12/1983,07681115087,3 Canary Avenue
6,176916326,33632099,2023-02-12,1955.6,18185362,Basic,2023-01-31,609.36,Agace Sporle,28/02/1999,07182359874,96 Emmet Pass
7,184329213,79033058,2023-02-04,1454.9,23231848,Gold,2023-01-31,27.12,Fee Moore,05/04/1944,07116175185,30 Aberg Way
9,202263606,22035411,2023-02-02,1028.8,99760030,Gold,2023-01-31,988.00,Mata Brownett,16/07/1931,07314541365,5 Kenwood Park
11,213599996,84448752,2023-02-08,1368.6,22137086,Basic,2023-01-31,410.35,Kerry Gaze,29/06/1992,07903643984,3 Esker Place
12,4025712466,72784766,2023-02-09,1288.6,22137086,Basic,2023-01-31,410.35,Kerry Gaze,29/06/1992,07903643984,3 Esker Place
13,215959175,64065043,2023-02-13,1619.8,55986027,Basic,2023-01-31,38.94,Honey Marns,08/04/1956,07121727227,637 Schmedeman Alley


End of Code